In [1]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /private/var/folders/f8/xh6w51d525g30x00m8swhsjh0000gn/T/pip-req-build-m_zb0wyw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/f8/xh6w51d525g30x00m8swhsjh0000gn/T/pip-req-build-m_zb0wyw
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ===== IMPORTS =====
import markdown
from bs4 import BeautifulSoup
import re
from pathlib import Path
from typing import Dict, Any, List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from pymilvus import MilvusClient, DataType, Function, FunctionType, AnnSearchRequest, RRFRanker
from PIL import Image
import torch
from tqdm import tqdm
import clip

In [3]:
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# ===== MODEL INITIALIZATION =====
# Text embedding model
text_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Multimodal embedding model (CLIP cho text-image joint embedding)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# Image captioning model (InternVL3.5-2B-Instruct)
# caption_tokenizer = AutoTokenizer.from_pretrained("OpenGVLab/InternVL3_5-2B-Instruct", trust_remote_code=True)
# caption_model = AutoModel.from_pretrained("OpenGVLab/InternVL3_5-2B-Instruct", trust_remote_code=True)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# ===== MARKDOWN PROCESSOR =====
class MarkdownProcessor:
    """Xử lý file markdown, trích xuất text, bảng, và hình ảnh"""

    def __init__(self, markdown_dir: str, image_base_dir: str):
        self.markdown_dir = Path(markdown_dir)
        self.image_base_dir = Path(image_base_dir)

    def extract_content(self, md_file: Path) -> Dict[str, Any]:
        """Trích xuất text, tables, và images từ markdown"""
        with open(md_file, 'r', encoding='utf-8') as f:
            md_content = f.read()

        html = markdown.markdown(md_content, extensions=['tables', 'fenced_code'])
        soup = BeautifulSoup(html, 'html.parser')

        text_content = soup.get_text(separator='\n', strip=True)

        tables = []
        for table in soup.find_all('table'):
            table_text = table.get_text(separator=' | ', strip=True)
            tables.append(table_text)

        image_paths = []
        img_pattern = r'!\[.*?\]\((.*?)\)'
        for match in re.finditer(img_pattern, md_content):
            img_path = match.group(1).strip()
            if img_path.startswith("images/"):
                full_path = self.markdown_dir / img_path
            else:
                full_path = self.image_base_dir / img_path

            if full_path.exists():
                image_paths.append(str(full_path))

        return {
            'text': text_content,
            'tables': tables,
            'images': image_paths,
            'source': str(md_file)
        }

In [ ]:
class MultimodalImageEncoder:
    """Encode images and text with CLIP embeddings"""

    def __init__(self, clip_model, clip_preprocess):
        self.clip_model = clip_model
        self.clip_preprocess = clip_preprocess
        self.device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

        # Move model to device
        self.clip_model = self.clip_model.to(self.device)

    def encode_image_multimodal(self, image_path: str) -> List[float]:
        """Encode image into CLIP embedding vector"""
        try:
            image = Image.open(image_path).convert('RGB')
            image_input = self.clip_preprocess(image).unsqueeze(0).to(self.device)

            with torch.no_grad():
                image_features = self.clip_model.encode_image(image_input)
                image_features /= image_features.norm(dim=-1, keepdim=True)

            return image_features.squeeze().cpu().numpy().tolist()
        except Exception as e:
            print(f"Error encoding image {image_path}: {e}")
            # Return zero vector of appropriate dimension
            return [0.0] * 512  # CLIP's standard dimension

    def encode_text_for_image_search(self, text: str) -> List[float]:
        """Encode text into CLIP embedding vector for text-to-image search"""
        try:
            text_tokens = clip.tokenize(text).to(self.device)

            with torch.no_grad():
                text_features = self.clip_model.encode_text(text_tokens)
                text_features /= text_features.norm(dim=-1, keepdim=True)

            return text_features.squeeze().cpu().numpy().tolist()
        except Exception as e:
            print(f"Error encoding text '{text}': {e}")
            # Return zero vector of appropriate dimension
            return [0.0] * 512  # CLIP's standard dimension
image_encoder = MultimodalImageEncoder(clip_model, clip_preprocess)

In [ ]:
# ===== ENTITY PROCESSORS =====
class TextEntityProcessor:
    """Xử lý entity text riêng biệt"""

    def __init__(self, text_model, chunk_size=500, overlap=100):
        self.text_model = text_model
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap
        )

    def create_text_entities(self, content: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Tạo text entities"""
        full_text = content['text']

        # Thêm tables vào text nếu có
        if content['tables']:
            full_text += "\n\n" + "\n\n".join(content['tables'])

        text_chunks = self.text_splitter.create_documents([full_text])

        entities = []
        for idx, chunk in enumerate(text_chunks):
            # Text embedding
            text_embedding = self.text_model.encode(chunk.page_content, convert_to_tensor=False).tolist()

            entity = {
                # 'id': f"text_{idx}",
                'content': chunk.page_content,
                'text_dense': text_embedding,  # Dense vector cho semantic search
                'text_sparse': chunk.page_content,  # Text cho BM25 sparse search
                'metadata': {
                    'entity_type': 'text',
                    'source': content['source'],
                    'chunk_index': idx,
                    'content_type': 'text_with_tables' if content['tables'] else 'text_only'
                },
                # Các trường image để trống
                'image_path': '',
                # 'image_caption': '',
                'image_dense': [0.0] * 512  # Vector trống với đúng dimension
            }
            entities.append(entity)

        return entities

In [ ]:
class ImageEntityProcessor:
    """Xử lý entity image riêng biệt"""

    def __init__(self, image_encoder):
        self.image_encoder = image_encoder
    
    def create_image_entities(self, content: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Tạo image entities"""
        entities = []

        for idx, img_path in enumerate(content['images']):
            # Generate caption
            # caption = self.image_encoder.generate_caption(img_path)

            # Multimodal embedding
            image_embedding = self.image_encoder.encode_image_multimodal(img_path)

            # Text embedding của caption (cho text search)
            # caption_embedding = text_model.encode(caption, convert_to_tensor=False).tolist()

            entity = {
                # 'id': f"image_{idx}",
                'content': "",  # Caption làm content chính
                'image_path': img_path,
                # 'image_caption': caption,
                'image_dense': image_embedding,  # Multimodal vector cho image search
                'metadata': {
                    'entity_type': 'image',
                    'source': content['source'],
                    'image_index': idx,
                    'original_path': img_path
                },
                # Các trường text bỏ trống cho image
                'text_dense': [0.0] * 768,  # Dense vector của caption
                # 'text_sparse': caption  # Caption text cho BM25
            }
            entities.append(entity)

        return entities

In [9]:
text_processor = TextEntityProcessor(text_model)
image_processor = ImageEntityProcessor(image_encoder)

In [ ]:

# ===== MILVUS HYBRID STORE =====
class MilvusHybridStore:
    """Milvus store tối ưu cho hybrid search"""

    def __init__(self, uri, token, collection_name="create_hybrid_pipeline_2"):
        self.client = MilvusClient(uri=uri, token=token)
        self.collection_name = collection_name
        self.text_dense_dim = 768  # paraphrase-multilingual-mpnet-base-v2
        self.image_dense_dim = 512  # clip

    def create_hybrid_collection(self):
        """Tạo collection cho hybrid search"""
        if self.client.has_collection(self.collection_name):
            self.client.drop_collection(self.collection_name)

        schema = self.client.create_schema(
            auto_id=True,
            enable_dynamic_fields=True
        )

        # Primary key
        schema.add_field(
            field_name="id",
            datatype=DataType.VARCHAR,
            is_primary=True,
            max_length=100
        )

        # Content field
        schema.add_field(
            field_name="content",
            datatype=DataType.VARCHAR,
            max_length=8000,
            enable_analyzer=True,
        )

        # Text fields
        schema.add_field(
            field_name="text_dense",
            datatype=DataType.FLOAT_VECTOR,
            dim=self.text_dense_dim,
        )

        schema.add_field(
            field_name="text_sparse",
            datatype=DataType.SPARSE_FLOAT_VECTOR,
        )

        # Image fields
        schema.add_field(
            field_name="image_dense",
            datatype=DataType.FLOAT_VECTOR,
            dim=self.image_dense_dim,
        )

        schema.add_field(
            field_name="image_path",
            datatype=DataType.VARCHAR,
            max_length=500,
        )

        # schema.add_field(
        #     field_name="image_caption",
        #     datatype=DataType.VARCHAR,
        #     max_length=1000,
        # )

        # Metadata
        schema.add_field(
            field_name="metadata",
            datatype=DataType.JSON
        )

        # BM25 function cho text sparse
        bm25_function = Function(
            name="text_bm25_emb",
            function_type=FunctionType.BM25,
            input_field_names=["content"],
            output_field_names=["text_sparse"]
        )
        schema.add_function(bm25_function)

        # Index parameters
        index_params = self.client.prepare_index_params()

        # Text dense index
        index_params.add_index(
            field_name="text_dense",
            index_type="HNSW",
            metric_type="COSINE",
            params={"M": 16, "efConstruction": 200}
        )

        # Text sparse index
        index_params.add_index(
            field_name="text_sparse",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="BM25"
        )

        # Image dense index
        index_params.add_index(
            field_name="image_dense",
            index_type="HNSW",
            metric_type="COSINE",
            params={"M": 16, "efConstruction": 200}
        )

        self.client.create_collection(
            collection_name=self.collection_name,
            schema=schema,
            index_params=index_params
        )

        print(f"✅ Hybrid collection '{self.collection_name}' đã được tạo!")

    def insert_entities(self, entities: List[Dict[str, Any]]):
        """Insert entities vào collection"""
        batch_size = 50
        for i in range(0, len(entities), batch_size):
            batch = entities[i:i + batch_size]

            # Remove text_sparse from entities as it's generated by BM25 function
            for entity in batch:
                entity.pop('text_sparse', None)

            self.client.insert(
                collection_name=self.collection_name,
                data=batch
            )
            print(f"✅ Đã insert batch {i//batch_size + 1}/{(len(entities)-1)//batch_size + 1}")

        print(f"✅ Đã insert {len(entities)} entities vào Milvus!")

In [ ]:
from pymilvus import AnnSearchRequest, RRFRanker

class HybridSearchEngine:
    """Engine cho hybrid search với multiple vectors từ text input"""

    def __init__(self, milvus_store, text_model, image_encoder):
        self.store = milvus_store
        self.text_model = text_model
        self.image_encoder = image_encoder

    def hybrid_search(self, query_text: str, limit: int = 10) -> List[Dict]:
        """Thực hiện hybrid search chỉ với text input"""

        search_requests = []

        if not query_text:
            return []

        # 1. Text semantic search với text embedding model
        query_text_embedding = self.text_model.encode(query_text, convert_to_tensor=False).tolist()

        text_search = AnnSearchRequest(
            data=[query_text_embedding],
            anns_field="text_dense",
            param={"nprobe": 10},
            limit=limit
        )
        search_requests.append(text_search)

        # 2. Full-text search (BM25) với sparse vectors
        sparse_search = AnnSearchRequest(
            data=[query_text],
            anns_field="text_sparse",
            param={"drop_ratio_search": 0.2},
            limit=limit
        )
        search_requests.append(sparse_search)

        # 3. Multimodal search - embedding text bằng image encoder
        query_image_embedding = self.image_encoder.encode_text_for_image_search(query_text)

        image_search = AnnSearchRequest(
            data=[query_image_embedding],
            anns_field="image_dense",
            param={"nprobe": 10},
            limit=limit
        )
        search_requests.append(image_search)

        # Thực hiện hybrid search với RRF ranker
        ranker = RRFRanker(100)

        results = self.store.client.hybrid_search(
            collection_name=self.store.collection_name,
            reqs=search_requests,
            ranker=ranker,
            limit=limit,
            output_fields=["content", "metadata", "image_path"]
        )

        return results[0] if results else []

In [ ]:
# ===== MAIN PIPELINE =====
def create_hybrid_pipeline(markdown_dir: str, image_base_dir: str,
                          uri: str, token: str):
    """Tạo pipeline hybrid search hoàn chỉnh"""

    # Initialize components
    processor = MarkdownProcessor(markdown_dir, image_base_dir)
    store = MilvusHybridStore(uri, token)
    search_engine = HybridSearchEngine(store, text_model, image_encoder)

    # Tạo collection
    store.create_hybrid_collection()

    def process_documents(markdown_files: List[str]):
        """Xử lý documents thành entities"""
        all_entities = []

        for md_file in markdown_files:
            print(f"\n{'='*60}")
            print(f"Đang xử lý: {md_file}")
            print(f"{'='*60}")

            # Extract content
            content = processor.extract_content(Path(md_file))
            print(f"  - Text length: {len(content['text'])} ký tự")
            print(f"  - Số bảng: {len(content['tables'])}")
            print(f"  - Số hình ảnh: {len(content['images'])}")

            # Tạo text entities
            text_entities = text_processor.create_text_entities(content)
            print(f"  - Text entities: {len(text_entities)}")

            # Tạo image entities
            image_entities = image_processor.create_image_entities(content)
            print(f"  - Image entities: {len(image_entities)}")

            all_entities.extend(text_entities)
            all_entities.extend(image_entities)

                # Insert vào Milvus
        store.insert_entities(all_entities)

        print(f"\n✅ Pipeline hoàn tất! Tổng cộng {len(all_entities)} entities đã được xử lý.")
        return all_entities, search_engine

    return process_documents, search_engine

In [13]:
# Configuration
MARKDOWN_DIR = 'test_data'
IMAGE_BASE_DIR = 'test_data/images'
MILVUS_URI = "https://in03-820213f77a18314.serverless.aws-eu-central-1.cloud.zilliz.com"
MILVUS_TOKEN = "3e35fd5ea0bdf741db327f1e34058b16a3f4634904e7fc717ca74238836e0fb9ef0aee7727d97a9276e46281ec07399bdc03f38e"
# MILVUS_URI = "điền uri vào đây"
# MILVUS_TOKEN = "điền token vào đây"

In [14]:
# Tạo pipeline
process_docs, search_engine = create_hybrid_pipeline(
     MARKDOWN_DIR, IMAGE_BASE_DIR, MILVUS_URI, MILVUS_TOKEN
)

✅ Hybrid collection 'hybrid_rag_collection' đã được tạo!


In [15]:
# Xử lý documents
markdown_files = list(Path(MARKDOWN_DIR).glob("*.md"))
entities, search_engine = process_docs(markdown_files)


Đang xử lý: test_data/Public048.md
  - Text length: 5734 ký tự
  - Số bảng: 4
  - Số hình ảnh: 11
  - Text entities: 16
  - Image entities: 11

Đang xử lý: test_data/Public043.md
  - Text length: 9891 ký tự
  - Số bảng: 7
  - Số hình ảnh: 14
  - Text entities: 28
  - Image entities: 14
✅ Đã insert batch 1/2
✅ Đã insert batch 2/2
✅ Đã insert 69 entities vào Milvus!

✅ Pipeline hoàn tất! Tổng cộng 69 entities đã được xử lý.


In [16]:
from pprint import pprint

In [ ]:
# print(f"\n{'='*60}")
# print("TEST HYBRID SEARCH")
# print(f"{'='*60}")

# Text search
text_results = search_engine.hybrid_search(
    query_text="kernel là gì ?",
    limit=5
)
# print(f"Text search results: {len(text_results)}")
# for i, result in enumerate(text_results):
#     pprint(f"  {i+1}. Score: {result['distance']:.4f}")
#     pprint(f"     Content: {result['entity']['content']}...")
#     pprint(f"     Type: {result['entity']['metadata']}")

In [18]:
pprint(text_results)

[{'id': '461027867908063963', 'distance': 0.01951637491583824, 'entity': {'content': '2. Phép tính convolution\n2.1 Convolution\nĐể cho dễ hình dung tôi sẽ lấy ví dụ trên ảnh xám, tức là ảnh được biểu diễn dưới dạng ma trận A kích thước $\\mathfrak { m } ^ { * } \\mathfrak { n }$ .\nTa định nghĩa kernel là một ma trận vuông kích thước $\\mathrm { k ^ { * } k }$ trong đó k là số lẻ. k có thể bằng 1, 3, 5, 7, 9,... Ví dụ kernel kích thước $3 ^ { * } 3$\n$$\nW = { \\left[ \\begin{array} { l l l } { 1 } & { 0 } & { 1 } \\ { 0 } & { 1 } & { 0 } \\ { 1 } & { 0 } & { 1 } \\end{array} \\right] }\n$$', 'image_path': '', 'metadata': {'entity_type': 'text', 'source': 'test_data/Public043.md', 'chunk_index': 16, 'content_type': 'text_with_tables'}}}, {'id': '461027867908063950', 'distance': 0.01913919486105442, 'entity': {'content': 'Bạn sẽ thấy chiều dài ảnh là 800 pixels (viết tắt px), chiều rộng 600 pixels, kích thước là $8 0 0 ^ { * } 6 0 0$ . Trước giờ chỉ học đơn vị đo là mét hay centimet, p

### Load câu hỏi

In [20]:
from transformers import AutoProcessor, AutoModelForVision2Seq

In [ ]:
import pandas as pd
from typing import List, Dict
from PIL import Image
import torch
import os

# Giả sử search_engine.hybrid_search() đã được định nghĩa
# Nếu không, bạn cần cung cấp chi tiết để tích hợp

def load_questions_from_csv(csv_path: str) -> List[Dict]:
    """
    Đọc câu hỏi và đáp án từ file CSV.
    CSV có các cột: Question, A, B, C, D
    """
    df = pd.read_csv(csv_path)
    questions = []
    for _, row in df.iterrows():
        question_data = {
            'question': row['Question'],
            'options': {
                'A': row['A'],
                'B': row['B'],
                'C': row['C'],
                'D': row['D']
            }
        }
        questions.append(question_data)
    return questions

def perform_vector_search(search_engine, query: str, k: int = 5) -> List[Dict]:
    """
    Thực hiện vector search cho một query, trả về top k kết quả.
    """
    return search_engine.hybrid_search(query_text=query, limit=k)

def collect_contexts(search_engine, question: str, options: Dict[str, str], k: int = 5) -> List[Dict]:
    """
    Thực hiện vector search cho câu hỏi và từng đáp án, gộp context.
    """
    contexts = []

    # Search cho câu hỏi
    question_results = perform_vector_search(search_engine, question, k)
    contexts.extend(question_results)

    # Search cho từng đáp án
    for option_key, option_text in options.items():
        option_results = perform_vector_search(search_engine, option_text, k)
        contexts.extend(option_results)

    return contexts

def format_contexts(contexts: List[Dict]) -> str:
    """
    Định dạng context (text và ảnh) thành chuỗi để đưa vào prompt.
    """
    formatted_context = ""
    for idx, ctx in enumerate(contexts, 1):
        content = ctx['entity'].get('content', '')
        image_path = ctx['entity'].get('image_path', '')
        distance = ctx['distance']
        source = ctx['entity']['metadata'].get('source', 'unknown')

        formatted_context += f"Context {idx} (Source: {source}, Distance: {distance:.4f}):\n"
        if content:
            formatted_context += f"Text: {content}\n"
        if image_path:
            formatted_context += f"Image: {image_path}\n"
        formatted_context += "\n"

    return formatted_context

def create_prompt(question: str, options: Dict[str, str], contexts: str) -> str:
    """
    Tạo prompt cho LLM dựa trên câu hỏi, đáp án và context.
    """
    prompt = f"""
Bạn là một trợ lý AI thông minh, được cung cấp context từ vector search để trả lời câu hỏi trắc nghiệm. Dựa trên context, hãy chọn đáp án đúng (có thể có nhiều đáp án đúng) từ các lựa chọn A, B, C, D. Trả về kết quả dưới dạng: "Đáp án đúng: [A, B, C, D]".

**Câu hỏi**: {question}

**Các đáp án**:
A: {options['A']}
B: {options['B']}
C: {options['C']}
D: {options['D']}

**Context**:
{contexts}

**Yêu cầu**:
- Phân tích context (bao gồm text và ảnh nếu có) để xác định đáp án đúng.
- Nếu context chứa ảnh, hãy mô tả cách ảnh liên quan đến câu hỏi (nếu có thể).
- Trả về đáp án đúng dưới dạng: "Đáp án đúng: [A, B, C, D]".
- Nếu không đủ thông tin, hãy trả về: "Không đủ thông tin để xác định đáp án đúng."
"""
    return prompt

def initialize_model(model_name: str = "Qwen/Qwen2.5-VL-3B-Instruct"):
    """
    Khởi tạo model Qwen2.5-VL-3B-Instruct và processor.
    """
    # Load processor (handles both text and image tokenization)
    processor = AutoProcessor.from_pretrained(model_name)

    # Load model
    model = AutoModelForVision2Seq.from_pretrained(model_name)

    return model, processor

def generate_response(model, processor, prompt: str, image_paths: List[str] = None, max_new_tokens: int = 512):
    """
    Gọi model Qwen2.5-VL-3B-Instruct để tạo phản hồi từ prompt và ảnh (nếu có).
    """
    # Chuẩn bị conversation
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ]
        }
    ]

    # Thêm ảnh vào conversation nếu có
    if image_paths:
        for img_path in image_paths:
            try:
                image = Image.open(img_path).convert("RGB")
                conversation[0]["content"].append({"type": "image", "image": image})
            except Exception as e:
                print(f"Không thể load ảnh {img_path}: {e}")

    # Áp dụng template và tokenize
    text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt],
        images=[content["image"] for content in conversation[0]["content"] if content["type"] == "image"] or None,
        return_tensors="pt"
    ).to(model.device)

    # Tạo phản hồi từ model
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # Không sử dụng sampling để đảm bảo kết quả chính xác
        num_beams=5,      # Beam search để cải thiện chất lượng
    )

    # Decode phản hồi
    response = processor.decode(outputs[0], skip_special_tokens=True)
    return response

def rag_pipeline(csv_path: str, search_engine):
    """
    Pipeline RAG hoàn chỉnh: đọc câu hỏi, vector search, gọi model, trả kết quả.
    """
    # Khởi tạo model và processor
    model, processor = initialize_model("Qwen/Qwen2.5-VL-3B-Instruct")

    # Đọc câu hỏi từ CSV
    questions = load_questions_from_csv(csv_path)
    results = []

    for q_data in questions:
        question = q_data['question']
        options = q_data['options']

        # Thu thập context từ vector search
        contexts = collect_contexts(search_engine, question, options)

        # Lấy danh sách đường dẫn ảnh từ context
        image_paths = [ctx['entity'].get('image_path', '') for ctx in contexts if ctx['entity'].get('image_path')]
        image_paths = [path for path in image_paths if path]  # Loại bỏ các path rỗng

        # Định dạng context
        formatted_contexts = format_contexts(contexts)

        # Tạo prompt
        prompt = create_prompt(question, options, formatted_contexts)

        # Gọi model để suy luận
        response = generate_response(model, processor, prompt, image_paths)

        results.append({
            'question': question,
            'options': options,
            'response': response
        })

    return results

# Ví dụ sử dụng
if __name__ == "__main__":
    csv_path = "training_input/question.csv"
    # search_engine: đối tượng search engine của bạn
    # Ví dụ: search_engine = YourSearchEngineClass()

    results = rag_pipeline(csv_path, search_engine)

    # In kết quả
    for result in results:
        print(f"Câu hỏi: {result['question']}")
        print(f"Đáp án: {result['options']}")
        print(f"Kết quả: {result['response']}")
        print("-" * 50)

/opt/anaconda3/envs/demo/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


: 